# Limpeza dos Dados

Etapa de limpeza dos dados originais a fim de manter apenas os registros e atributos necessárias para as etapas de análsie exploratória e aplicação dos modelos.

In [3]:
# Bibliotecas de manipulação de dados
import pandas as pd
import numpy as np

# Bibliotecas para encontrar a latitue e longitude de cidades
from geopy.geocoders import Nominatim
import brazilcep

# Biblioteca para desligar os avisos do Python
import warnings

# Bibliotecas para aplicar o Robust Scaling
from sklearn.preprocessing import RobustScaler
from statsmodels.robust.scale import mad as median_abs_deviation

# Biblioteca para auxiliar na transformação das datas em valores numéricos
from datetime import datetime

# Igonorar os avisos e imprimir todas as colunas
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [4]:
# Imprimir informações sobre os dados originais
df = pd.read_csv(r'..\..\datasets\nfs_original.csv', encoding='latin-1')
pd.set_option('display.max_columns', None)
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2089317 entries, 0 to 2089316
Data columns (total 62 columns):
 #   Column                                Non-Null Count    Dtype  
---  ------                                --------------    -----  
 0   Numero                                2089317 non-null  int64  
 1   Data_de_emissao                       2089317 non-null  object 
 2   Situacao                              2089317 non-null  object 
 3   Valor_total_da_nota                   2089317 non-null  float64
 4   Nota_referenciada                     160015 non-null   object 
 5   Nome_razao_social_emit                2089317 non-null  object 
 6   CPF_CNPJ_emit                         2089317 non-null  int64  
 7   Endereco_emit                         2089317 non-null  object 
 8   Bairro_distrito_emit                  2089314 non-null  object 
 9   CEP_emit                              2089317 non-null  float64
 10  Municipio_emit                        2089317 non-null

## 01 Retirada de atributos irrelevantes

Após a análise de cada coluna, foram identificadas colunas que não seriam adequadas para o problema. 
Os principais motivos foram: (i) grande quantidade de valores nulos e (ii) informações irrelevantes ou confusas

In [5]:
# Retira colunas irrelevantes
df_cleaned = df[['Numero',
                 'Nr_item',
                 'Data_de_emissao',
                 'Valor_total_da_nota',
                 'Nome_razao_social_emit',
                 'CPF_CNPJ_emit',
                 'Bairro_distrito_emit',
                 'Municipio_emit',
                 'CEP_emit',
                 'Descricao_do_Produto_ou_servicos',
                 'NCM_prod',
                 'CFOP_prod',
                 'Quant_prod',
                 'Unid_prod',
                 'Valor_unit_prod',
                 'Valor_total_prod']]
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2089317 entries, 0 to 2089316
Data columns (total 16 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   Numero                            int64  
 1   Nr_item                           int64  
 2   Data_de_emissao                   object 
 3   Valor_total_da_nota               float64
 4   Nome_razao_social_emit            object 
 5   CPF_CNPJ_emit                     int64  
 6   Bairro_distrito_emit              object 
 7   Municipio_emit                    object 
 8   CEP_emit                          float64
 9   Descricao_do_Produto_ou_servicos  object 
 10  NCM_prod                          float64
 11  CFOP_prod                         float64
 12  Quant_prod                        float64
 13  Unid_prod                         object 
 14  Valor_unit_prod                   float64
 15  Valor_total_prod                  float64
dtypes: float64(7), int64(3), object(6)
m

In [6]:
# Altera os nomes das colunas
new_column_names = {'Numero': 'nf_numero', 
                    'Nr_item': 'nf_item',
                    'Data_de_emissao': 'nf_datahora',
                    'Valor_total_da_nota': 'nf_valor_total',
                    'Nome_razao_social_emit': 'emit_nome',
                    'CPF_CNPJ_emit': 'emit_cnpj',
                    'Bairro_distrito_emit': 'emit_bairro',
                    'Municipio_emit': 'emit_municipio',
                    'CEP_emit': 'emit_cep',
                    'Descricao_do_Produto_ou_servicos': 'prod_desc',
                    'NCM_prod': 'prod_ncm',
                    'CFOP_prod': 'prod_cfop',
                    'Quant_prod': 'prod_quant',
                    'Unid_prod': 'prod_unid',
                    'Valor_unit_prod': 'prod_valor_unit',
                    'Valor_total_prod': 'prod_valor_total'}

df_cleaned.rename(columns=new_column_names, inplace=True)
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2089317 entries, 0 to 2089316
Data columns (total 16 columns):
 #   Column            Dtype  
---  ------            -----  
 0   nf_numero         int64  
 1   nf_item           int64  
 2   nf_datahora       object 
 3   nf_valor_total    float64
 4   emit_nome         object 
 5   emit_cnpj         int64  
 6   emit_bairro       object 
 7   emit_municipio    object 
 8   emit_cep          float64
 9   prod_desc         object 
 10  prod_ncm          float64
 11  prod_cfop         float64
 12  prod_quant        float64
 13  prod_unid         object 
 14  prod_valor_unit   float64
 15  prod_valor_total  float64
dtypes: float64(7), int64(3), object(6)
memory usage: 255.0+ MB


## 02 Retirada dos registros contendo valores nulos e duplicados

In [7]:
# Retira registros que contém valores nulos
df_cleaned.dropna(inplace=True)

# Retira registros duplicados
df_cleaned.drop_duplicates()

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2067051 entries, 0 to 2089316
Data columns (total 16 columns):
 #   Column            Dtype  
---  ------            -----  
 0   nf_numero         int64  
 1   nf_item           int64  
 2   nf_datahora       object 
 3   nf_valor_total    float64
 4   emit_nome         object 
 5   emit_cnpj         int64  
 6   emit_bairro       object 
 7   emit_municipio    object 
 8   emit_cep          float64
 9   prod_desc         object 
 10  prod_ncm          float64
 11  prod_cfop         float64
 12  prod_quant        float64
 13  prod_unid         object 
 14  prod_valor_unit   float64
 15  prod_valor_total  float64
dtypes: float64(7), int64(3), object(6)
memory usage: 268.1+ MB


## 03 Engenharia de atributos

Etapa de transformação dos dados para que sejam úteis para a aplicação dos modelos

In [8]:
# Faz o casting dos valores das colunas para os tipos de dados corretos
df_cleaned['nf_datahora'] = pd.to_datetime(df_cleaned['nf_datahora'])
df_cleaned['emit_nome'] = df_cleaned['emit_nome'].str.lower()
df_cleaned['emit_bairro'] = df_cleaned['emit_bairro'].str.lower()
df_cleaned['emit_municipio'] = df_cleaned['emit_municipio'].str.lower()
df_cleaned['emit_cep'] = df_cleaned['emit_cep'].astype('int')
df_cleaned['prod_desc'] = df_cleaned['prod_desc'].str.lower()
df_cleaned['prod_ncm'] = df_cleaned['prod_ncm'].astype('int')
df_cleaned['prod_cfop'] = df_cleaned['prod_cfop'].astype('int')
df_cleaned['prod_unid'] = df_cleaned['prod_unid'].str.lower()

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2067051 entries, 0 to 2089316
Data columns (total 16 columns):
 #   Column            Dtype         
---  ------            -----         
 0   nf_numero         int64         
 1   nf_item           int64         
 2   nf_datahora       datetime64[ns]
 3   nf_valor_total    float64       
 4   emit_nome         object        
 5   emit_cnpj         int64         
 6   emit_bairro       object        
 7   emit_municipio    object        
 8   emit_cep          int32         
 9   prod_desc         object        
 10  prod_ncm          int32         
 11  prod_cfop         int32         
 12  prod_quant        float64       
 13  prod_unid         object        
 14  prod_valor_unit   float64       
 15  prod_valor_total  float64       
dtypes: datetime64[ns](1), float64(4), int32(3), int64(3), object(5)
memory usage: 244.4+ MB


In [9]:
df_cleaned.head(5)

,nf_numero,nf_item,nf_datahora,nf_valor_total,emit_nome,emit_cnpj,emit_bairro,emit_municipio,emit_cep,prod_desc,prod_ncm,prod_cfop,prod_quant,prod_unid,prod_valor_unit,prod_valor_total
0,8326,1,2016-02-04 14:30:38,45.99,leia comercial de livros e magazine ltda,11895653000100,centro,cajazeiras,58900000,grampo plast dello fix bc - 0301.e0020.2 - delo,39261000,5929,1.0,und,9.95,9.95
1,8326,2,2016-02-04 14:30:38,45.99,leia comercial de livros e magazine ltda,11895653000100,centro,cajazeiras,58900000,lapiseira tecnica npy 0.9 - 14088 - molin,96084000,5929,1.0,und,8.30,8.30
2,8326,3,2016-02-04 14:30:38,45.99,leia comercial de livros e magazine ltda,11895653000100,centro,cajazeiras,58900000,lapiseira tec job 0.7 - 14077 - molin,96084000,5929,1.0,und,6.95,6.95
3,8326,4,2016-02-04 14:30:38,45.99,leia comercial de livros e magazine ltda,11895653000100,centro,cajazeiras,58900000,marca texto fluo peps amarelo - 742534 - maped,96082000,5929,1.0,und,3.70,3.70
4,8326,5,2016-02-04 14:30:38,45.99,leia comercial de livros e magazine ltda,11895653000100,centro,cajazeiras,58900000,grampeador 26/6 t320 12cm 20f preto - 665207 ...,84729040,5929,1.0,und,16.50,16.50


In [10]:
# Cria uma nova coluna contendo o tempo, em segundos, de cada valor da coluna de data a partir da Era Unix (1970-01-01)
df_cleaned['nf_timestamp'] = (df_cleaned['nf_datahora'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2067051 entries, 0 to 2089316
Data columns (total 17 columns):
 #   Column            Dtype         
---  ------            -----         
 0   nf_numero         int64         
 1   nf_item           int64         
 2   nf_datahora       datetime64[ns]
 3   nf_valor_total    float64       
 4   emit_nome         object        
 5   emit_cnpj         int64         
 6   emit_bairro       object        
 7   emit_municipio    object        
 8   emit_cep          int32         
 9   prod_desc         object        
 10  prod_ncm          int32         
 11  prod_cfop         int32         
 12  prod_quant        float64       
 13  prod_unid         object        
 14  prod_valor_unit   float64       
 15  prod_valor_total  float64       
 16  nf_timestamp      int64         
dtypes: datetime64[ns](1), float64(4), int32(3), int64(4), object(5)
memory usage: 260.2+ MB


In [11]:
# Remove os seguintes valoes da coluna emit_municipio
delete_list = ['2513703',
               '2507507',
               '2501807',
               '2508901',
               '2504009',
               'araeccedil',
               'riachao do poeccedil',
               'rua deputado francis',
               '<definir> 9089',
               'praca iniguacu',
               'cg',
               'rua rodrigues de aqu',
               'sao domingos de',
               'sao domingos do',
               'av. senador ruy carn',
               'araeccedil;agi',
               'riachao do poeccedil;o',
               'santa cecilia de umbuzeiro']

for item in delete_list:
    df_cleaned = df_cleaned[df_cleaned['emit_municipio'] != item]
    df_cleaned = df_cleaned[df_cleaned['emit_municipio'] != item]

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2066780 entries, 0 to 2089316
Data columns (total 17 columns):
 #   Column            Dtype         
---  ------            -----         
 0   nf_numero         int64         
 1   nf_item           int64         
 2   nf_datahora       datetime64[ns]
 3   nf_valor_total    float64       
 4   emit_nome         object        
 5   emit_cnpj         int64         
 6   emit_bairro       object        
 7   emit_municipio    object        
 8   emit_cep          int32         
 9   prod_desc         object        
 10  prod_ncm          int32         
 11  prod_cfop         int32         
 12  prod_quant        float64       
 13  prod_unid         object        
 14  prod_valor_unit   float64       
 15  prod_valor_total  float64       
 16  nf_timestamp      int64         
dtypes: datetime64[ns](1), float64(4), int32(3), int64(4), object(5)
memory usage: 260.2+ MB


In [12]:
# Lista de cidades que precisam ser renomeadas pois contém erros de gramática
replacement_list = [
    "sao jos ede piranhas -> sao jose de piranhas",
    "cuiteji -> cuitegi",
    "bayuex -> bayeux",
    "jaboatao dos grarapes -> jaboatao dos guararapes",
    "campo de santan -> campo de santana",
    "sum? -> sume",
    "campina grand -> campina grande",
    "sao vicentedo serido -> sao vicente do serido",
    "sao joao do car -> sao joao do caru",
    "ministro andrea -> ministro andreazza",
    "piloezinho -> piloezinhos",
    "alagoa grabde -> alagoa grande",
    "areia de baraun -> areia de baraunas",
    "juazerinho -> juazeirinho",
    "s jose do sabugy -> sao jose do sabugi",
    "s.j.dos cordeiros -> sao jose dos cordeiros",
    "araaagi -> aracagi",
    "sao joao do tig -> sao joao do tigre",
    "cruz do esp. santo -> cruz do espirito santo",
    "sao domingos de pomb -> sao domingos de pombal",
    "sao jose do bre -> sao jose do brejo do cruz",
    "são mamende -> são mamede",
    "sao seb de lagoa de -> sao sebastiao de lagoa de roca",
    "ara?agi -> aracagi",
    "riachao do baca -> riachao do bacamarte",
    "barra de santan -> barra de santana",
    "06 agua branca -> agua branca",
    "riacao do poco -> riachao do poco",
    "cacimba de arei -> cacimba de areia",
    "catol? do rocha -> catole do rocha",
    "santa cecilia de umbuzeiro -> santa cecilia de umbuzeiro",
    "sao jose de pri -> sao jose de princesa",
    "riacho de santo anto -> riacho de santo antonio",
    "riacho de santo antonio2512788 -> riacho de santo antonio",
    "salg. de sao felix -> salgado de são félix",
    "sao domingos do cari -> sao domingos do cariri",
    "sao jose de esp -> são josé de espinharas",
    "sao jose do bon -> sao jose do bonfim",
    "sao jose do bomfim -> sao jose do bonfim",
    "sao jose dos cordeir -> sao jose dos cordeiros",
    "sao minguel de taipu -> sao miguel de taipu",
    "sao sebastiao de lagoa de -> sao sebastiao de lagoa de roca",
    "riacho dos cavalo -> riacho dos cavalos",
    "joaopessoa -> joao pessoa",
    "joa o pessoa -> joao pessoa",
    "joao pessoa (pb) -> joao pessoa",
    "mae d,agua -> mae d-agua",
    "sao jose de espinhar -> sao jose de espinharas",
    "cruz do espirito san -> cruz do espirito santo",
    "cruz do espirit -> cruz do espirito santo",
    "sao sebastiao do umb -> sao sebastiao do umbuzeiro",
    "salgad -> salgado",
    "sao sebastiao do umbuzeir -> sao sebastiao do umbuzeiro",
    "sao seb. lagoa roca -> sao sebastiao de lagoa de roca",
    "sao sebast lagoa de roca -> sao sebastiao de lagoa de roca",
    "mirante da serr -> mirante da serra",
    "cacimba de dent -> cacimba de dentro",
    "catole do roch -> catole do rocha",
    "nava palmeira -> nova palmeira",
    "olho d,agua -> olho d-agua",
    "olho da agua -> olho d-agua",
    "poco de jose de mour -> poco de jose de moura",
    "caajzeiras -> cajazeiras",
    "serra bramca -> serra branca",
    "taberoa -> taperoa",
    "cabo de sto. agostin -> cabo de santo agostinho",
    "olho daagua -> olho d-agua",
    "pedrabranca -> pedra branca",
    "joao pessoaemp_uf=pb -> joao pessoa",
    "jab. dos guararapes -> jaboatao dos guararapes",
    "baia da trai??o -> baia da traicao",
    "floriana?polis -> florianopolis",
    "jo?o pessoa -> joao pessoa",
    "jaboatao dos guarara -> jaboatao dos guararapes",
    "so carlos -> sao carlos",
    "recefe -> recife",
    "algodao de jand -> algodao de jandaira",
    "mamamguape -> mamanguape",
    "joao pesso -> joao pessoa",
    "mae da agua -> mae d-agua",
]

# Renomeia as cidades necessárias
for replacement_rule in replacement_list:
    old_value, new_value = replacement_rule.split(' -> ')
    df_cleaned['emit_municipio'] = df_cleaned['emit_municipio'].replace(old_value, new_value)
    
for replacement_rule in replacement_list:
    old_value, new_value = replacement_rule.split(' -> ')
    df_cleaned['emit_municipio'] = df_cleaned['emit_municipio'].replace(old_value, new_value)
    
df_cleaned = df_cleaned.drop_duplicates()
df_cleaned = df_cleaned.dropna()
df_cleaned.reset_index(drop=True, inplace=True)
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2066754 entries, 0 to 2066753
Data columns (total 17 columns):
 #   Column            Dtype         
---  ------            -----         
 0   nf_numero         int64         
 1   nf_item           int64         
 2   nf_datahora       datetime64[ns]
 3   nf_valor_total    float64       
 4   emit_nome         object        
 5   emit_cnpj         int64         
 6   emit_bairro       object        
 7   emit_municipio    object        
 8   emit_cep          int32         
 9   prod_desc         object        
 10  prod_ncm          int32         
 11  prod_cfop         int32         
 12  prod_quant        float64       
 13  prod_unid         object        
 14  prod_valor_unit   float64       
 15  prod_valor_total  float64       
 16  nf_timestamp      int64         
dtypes: datetime64[ns](1), float64(4), int32(3), int64(4), object(5)
memory usage: 244.4+ MB


In [13]:
# Cria uma lista de municípios únicos da coluna emit_municipio
municipios = df_cleaned[['emit_municipio']].drop_duplicates()
municipios.rename(columns={'emit_municipio': 'municipio'}, inplace=True)
municipios = municipios.dropna()
municipios.reset_index(drop=True, inplace=True)
municipios

,municipio
0,cajazeiras
1,sousa
2,campina grande
3,joao pessoa
4,agua branca
...,...
654,itaúna
655,caetite
656,feliz
657,paulinia


In [14]:
# Utilizando a biblioteca GeoPy e brazilcep, encontra a latitute e longitude das cidades listadas acima
geolocator = Nominatim(user_agent="tcc")

def get_lat_long(cidade):
    query = f"{cidade}, brasil"
    print(query)
    try:
        location = geolocator.geocode(query, timeout=10)
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Timeout exception for {query}. Moving to the next item.")
    return None, None

municipios['lat'], municipios['long'] = zip(*municipios.apply(lambda row: get_lat_long(row['municipio']), axis=1))

cajazeiras, brasil
sousa, brasil
campina grande, brasil
joao pessoa, brasil
agua branca, brasil
aguiar, brasil
alagoa grande, brasil
alagoa nova, brasil
alagoinha, brasil
guarabira, brasil
alcantil, brasil
esperanca, brasil
alhandra, brasil
amparo, brasil
sume, brasil
ouro velho, brasil
aparecida, brasil
aracagi, brasil
sape, brasil
belo horizonte, brasil
joão pessoa, brasil
natal, brasil
arara, brasil
areia, brasil
queimadas, brasil
assuncao, brasil
recife, brasil
rio de janeiro, brasil
capim, brasil
fortaleza, brasil
mari, brasil
rio tinto, brasil
solanea, brasil
bananeiras, brasil
betim, brasil
timbauba, brasil
barra de santa rosa, brasil
bayeux, brasil
cabedelo, brasil
belem, brasil
belem do brejo do cruz, brasil
são bento, brasil
boqueirao, brasil
cabaceiras, brasil
cacimba de dentro, brasil
camalau, brasil
monteiro, brasil
parari, brasil
taperoa, brasil
araruna, brasil
patos, brasil
areial, brasil
barauna, brasil
cuite, brasil
diamante, brasil
bom sucesso, brasil
brejo do cruz, b

itu, brasil
lauro de freitas, brasil
feira de santana, brasil
marechal deodoro, brasil
santa barbara d'oeste, brasil
sao jose dos cordeiros, brasil
sao caetano do sul, brasil
varzea paulista, brasil
apodi, brasil
cascavel, brasil
rio claro, brasil
macau, brasil
louveira, brasil
são paulo, brasil
carapicuiba, brasil
mesquita, brasil
recife (pe), brasil
formiga, brasil
balneario camboriu, brasil
erechim, brasil
brasilia (gama), brasil
dores de campos, brasil
serra, brasil
eusebio, brasil
atibaia, brasil
aquiraz, brasil
sao jose do rio preto, brasil
uberaba, brasil
cambe, brasil
araraquara, brasil
belém do brejo do cruz, brasil
sao jose, brasil
taquarucu do sul, brasil
bauru, brasil
presidente prudente, brasil
brejo santo, brasil
paranaiba, brasil
olho d'agua, brasil
pinhais, brasil
leme, brasil
areia branca, brasil
municipio itupeva, brasil
nova lima, brasil
ilheus, brasil
picos, brasil
hortolandia, brasil
brumado, brasil
barro, brasil
sao luis, brasil
baependi, brasil
jacana, brasil
ita

In [15]:
municipios

,municipio,lat,long
0,cajazeiras,-6.889785,-38.557039
1,sousa,-6.766203,-38.231961
2,campina grande,-7.224674,-35.877129
3,joao pessoa,-7.121598,-34.882028
4,agua branca,-5.891392,-42.636115
...,...,...,...
654,itaúna,-20.075556,-44.576389
655,caetite,-14.069178,-42.485243
656,feliz,-29.452325,-51.306978
657,paulinia,-22.763039,-47.153221


In [17]:
municipios.to_csv(r'..\..\datasets\municipios.csv', index=False)

In [18]:
# Realiza um join (ou merge) entre os dados originais e os dados da lista contendo a latitude e longitude, 
# dessa forma foram criadas mais duas colunas contendo a latitude e longitude da cidade dos registros
df_cleaned = pd.merge(df_cleaned, municipios, left_on='emit_municipio', right_on='municipio')
df_cleaned.head(5)

,nf_numero,nf_item,nf_datahora,nf_valor_total,emit_nome,emit_cnpj,emit_bairro,emit_municipio,emit_cep,prod_desc,prod_ncm,prod_cfop,prod_quant,prod_unid,prod_valor_unit,prod_valor_total,nf_timestamp,municipio,lat,long
0,8326,1,2016-02-04 14:30:38,45.99,leia comercial de livros e magazine ltda,11895653000100,centro,cajazeiras,58900000,grampo plast dello fix bc - 0301.e0020.2 - delo,39261000,5929,1.0,und,9.95,9.95,1454596238,cajazeiras,-6.889785,-38.557039
1,8326,2,2016-02-04 14:30:38,45.99,leia comercial de livros e magazine ltda,11895653000100,centro,cajazeiras,58900000,lapiseira tecnica npy 0.9 - 14088 - molin,96084000,5929,1.0,und,8.30,8.30,1454596238,cajazeiras,-6.889785,-38.557039
2,8326,3,2016-02-04 14:30:38,45.99,leia comercial de livros e magazine ltda,11895653000100,centro,cajazeiras,58900000,lapiseira tec job 0.7 - 14077 - molin,96084000,5929,1.0,und,6.95,6.95,1454596238,cajazeiras,-6.889785,-38.557039
3,8326,4,2016-02-04 14:30:38,45.99,leia comercial de livros e magazine ltda,11895653000100,centro,cajazeiras,58900000,marca texto fluo peps amarelo - 742534 - maped,96082000,5929,1.0,und,3.70,3.70,1454596238,cajazeiras,-6.889785,-38.557039
4,8326,5,2016-02-04 14:30:38,45.99,leia comercial de livros e magazine ltda,11895653000100,centro,cajazeiras,58900000,grampeador 26/6 t320 12cm 20f preto - 665207 ...,84729040,5929,1.0,und,16.50,16.50,1454596238,cajazeiras,-6.889785,-38.557039


In [19]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2066754 entries, 0 to 2066753
Data columns (total 20 columns):
 #   Column            Dtype         
---  ------            -----         
 0   nf_numero         int64         
 1   nf_item           int64         
 2   nf_datahora       datetime64[ns]
 3   nf_valor_total    float64       
 4   emit_nome         object        
 5   emit_cnpj         int64         
 6   emit_bairro       object        
 7   emit_municipio    object        
 8   emit_cep          int32         
 9   prod_desc         object        
 10  prod_ncm          int32         
 11  prod_cfop         int32         
 12  prod_quant        float64       
 13  prod_unid         object        
 14  prod_valor_unit   float64       
 15  prod_valor_total  float64       
 16  nf_timestamp      int64         
 17  municipio         object        
 18  lat               float64       
 19  long              float64       
dtypes: datetime64[ns](1), float64(6), int32(3), in

In [20]:
# Renomeia as novas colunas
new_column_names = {'lat': 'emit_lat',
                    'long': 'emit_long'}

df_cleaned.rename(columns=new_column_names, inplace=True)
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2066754 entries, 0 to 2066753
Data columns (total 20 columns):
 #   Column            Dtype         
---  ------            -----         
 0   nf_numero         int64         
 1   nf_item           int64         
 2   nf_datahora       datetime64[ns]
 3   nf_valor_total    float64       
 4   emit_nome         object        
 5   emit_cnpj         int64         
 6   emit_bairro       object        
 7   emit_municipio    object        
 8   emit_cep          int32         
 9   prod_desc         object        
 10  prod_ncm          int32         
 11  prod_cfop         int32         
 12  prod_quant        float64       
 13  prod_unid         object        
 14  prod_valor_unit   float64       
 15  prod_valor_total  float64       
 16  nf_timestamp      int64         
 17  municipio         object        
 18  emit_lat          float64       
 19  emit_long         float64       
dtypes: datetime64[ns](1), float64(6), int32(3), in

In [21]:
# Remove a coluna 'municipio'
df_cleaned.drop(['municipio'], axis=1, inplace=True)
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2066754 entries, 0 to 2066753
Data columns (total 19 columns):
 #   Column            Dtype         
---  ------            -----         
 0   nf_numero         int64         
 1   nf_item           int64         
 2   nf_datahora       datetime64[ns]
 3   nf_valor_total    float64       
 4   emit_nome         object        
 5   emit_cnpj         int64         
 6   emit_bairro       object        
 7   emit_municipio    object        
 8   emit_cep          int32         
 9   prod_desc         object        
 10  prod_ncm          int32         
 11  prod_cfop         int32         
 12  prod_quant        float64       
 13  prod_unid         object        
 14  prod_valor_unit   float64       
 15  prod_valor_total  float64       
 16  nf_timestamp      int64         
 17  emit_lat          float64       
 18  emit_long         float64       
dtypes: datetime64[ns](1), float64(6), int32(3), int64(4), object(5)
memory usage: 291.7+

In [22]:
# Ordena as colunas
df_cleaned = df_cleaned[[
    'nf_numero',
    'nf_item',
    'nf_datahora',
    'nf_timestamp',
    'nf_valor_total',
    'emit_nome',
    'emit_cnpj',
    'emit_bairro',
    'emit_municipio',
    'emit_cep',
    'emit_lat',
    'emit_long',
    'prod_desc',
    'prod_ncm',
    'prod_cfop',
    'prod_quant',
    'prod_unid',
    'prod_valor_unit',
    'prod_valor_total',
]]

df_cleaned = df_cleaned.dropna()
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2065547 entries, 0 to 2066753
Data columns (total 19 columns):
 #   Column            Dtype         
---  ------            -----         
 0   nf_numero         int64         
 1   nf_item           int64         
 2   nf_datahora       datetime64[ns]
 3   nf_timestamp      int64         
 4   nf_valor_total    float64       
 5   emit_nome         object        
 6   emit_cnpj         int64         
 7   emit_bairro       object        
 8   emit_municipio    object        
 9   emit_cep          int32         
 10  emit_lat          float64       
 11  emit_long         float64       
 12  prod_desc         object        
 13  prod_ncm          int32         
 14  prod_cfop         int32         
 15  prod_quant        float64       
 16  prod_unid         object        
 17  prod_valor_unit   float64       
 18  prod_valor_total  float64       
dtypes: datetime64[ns](1), float64(6), int32(3), int64(4), object(5)
memory usage: 291.5+

## Registro da base

In [24]:
# Salva a base limpa
df_cleaned.to_csv(r'..\..\datasets\nfs_cleaned.csv', index=False)